# Notebook # 2

This is the second notebook for the `log-to-azure` example. In this notebook we load our resources and run the model.

For the sake of clarity all packages and modules are imported in the cells in which they are used.

# Load all assets.
Load in the assets we created in the previous notebook

In [1]:
from azureml.core import Workspace, Dataset, Environment

# Load the stored workspace
ws = Workspace.from_config()

# Get the registered dataset from azure
dataset = Dataset.get_by_name(ws, name='waste_images')

## Try with our saved image
env = Environment.get(workspace=ws, name="waste-env-gpu")

# get our compute target
compute_target = ws.compute_targets["gpu-cluster-NC6"]

## Define experiment name
An experiment is a container for one or more model runs.  You can group iterations of a model into an experiment for easy access on azureml Studio.  Every run must be in an experiment.

In [2]:
from azureml.core import Experiment

# define the expiriment
exp = Experiment(workspace=ws, name='recycling')

## Create train script
The train script is what will be executed on the azureml compute instance.  The directory specified will be uploaded and other assets or modules can be included in that directory.

In [3]:
import os

# create a directory for the training script
os.makedirs('train_script', exist_ok=True)

In [4]:
%%writefile train_script/train.py

import argparse
import os
import pickle

import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# FOR LOGGING TO AZURE
from log_to_azure import LogToAzure
from azureml.core import Run

# output will be logged, separate output from previous log entries.
print('-'*100)

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--data-path', type=str, 
                        dest='data_path', 
                        default='../data', 
                        help='data folder mounting point')

    return parser.parse_args()


if __name__ == '__main__':

    # parse the parameters passed to the this script
    args = parse_args()

    # set data paths
    train_folder = os.path.join(args.data_path, 'train')
    val_folder = os.path.join(args.data_path, 'validation')

    print(train_folder)

    # Create ImageGenerators
    print('Creating train ImageDataGenerator')
    train_generator = ImageDataGenerator(rescale=1/255)\
                            .flow_from_directory(train_folder, 
                                                 batch_size = 32)
    val_generator = ImageDataGenerator(rescale=1/255)\
                            .flow_from_directory(val_folder, 
                                                 batch_size = 32)

    # Build the model
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(32, (2,2), activation='relu'))
    model.add(keras.layers.MaxPooling2D(2,2))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(6, activation='softmax'))

    metrics = [keras.metrics.CategoricalAccuracy(),
               keras.metrics.Recall(),
               keras.metrics.Precision()]
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=metrics)

    
    # FOR LOGGING TO AZURE
    run = Run.get_context()
    callbacks = [LogToAzure(run)]
    
    # fit model and store history
    model.fit(train_generator, 
              validation_data=val_generator,
              callbacks=callbacks,
              epochs=10)

    print('Done!')
    print('-'*100)

Overwriting train_script/train.py


In [5]:
!cp ../log_to_azure.py train_script/

## Run the Model

In [6]:
from azureml.core import ScriptRunConfig

# setup the run details
src = ScriptRunConfig(source_directory='train_script',
                      script='train.py',
                      arguments=['--data-path', dataset.as_mount()],
                      compute_target=compute_target,
                      environment=env)

# Submit the model to azure!
run = exp.submit(config=src)

## Create a second model
This model is almost the same but with an added conv2d layer.  This is just so we can see different results for both models in the Azure ML Studio Dashboard.

In [7]:
%%writefile train_script/train.py

import argparse
import os
import pickle

import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# FOR LOGGING TO AZURE
from log_to_azure import LogToAzure
from azureml.core import Run

# output will be logged, separate output from previous log entries.
print('-'*100)

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--data-path', type=str, 
                        dest='data_path', 
                        default='../data', 
                        help='data folder mounting point')

    return parser.parse_args()


if __name__ == '__main__':

    # parse the parameters passed to the this script
    args = parse_args()

    # set data paths
    train_folder = os.path.join(args.data_path, 'train')
    val_folder = os.path.join(args.data_path, 'validation')

    print(train_folder)

    # Create ImageGenerators
    print('Creating train ImageDataGenerator')
    train_generator = ImageDataGenerator(rescale=1/255)\
                            .flow_from_directory(train_folder, 
                                                 batch_size = 32)
    val_generator = ImageDataGenerator(rescale=1/255)\
                            .flow_from_directory(val_folder, 
                                                 batch_size = 32)

    # Build the model
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(32, (2,2), activation='relu'))
    model.add(keras.layers.MaxPooling2D(2,2))
    model.add(keras.layers.Conv2D(16, (2,2), activation='relu'))
    model.add(keras.layers.MaxPooling2D(2,2))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(6, activation='softmax'))

    metrics = [keras.metrics.CategoricalAccuracy(),
               keras.metrics.Recall(),
               keras.metrics.Precision()]
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=metrics)

    
    # FOR LOGGING TO AZURE
    run = Run.get_context()
    callbacks = [LogToAzure(run)]
    
    # fit model and store history
    model.fit(train_generator, 
              validation_data=val_generator,
              callbacks=callbacks,
              epochs=10)

    print('Done!')
    print('-'*100)

Overwriting train_script/train.py


## Run the second model

In [8]:
from azureml.core import ScriptRunConfig

# setup the run details
src = ScriptRunConfig(source_directory='train_script',
                      script='train.py',
                      arguments=['--data-path', dataset.as_mount()],
                      compute_target=compute_target,
                      environment=env)

# Submit the model to azure!
run = exp.submit(config=src)